schedule on thursday

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

from datetime import datetime, timedelta
import pandas as pd
import glob
import os
import openpyxl

import sys
sys.path.append('../utils')
from SharePoint import *
from hlpr import *
from static import *

import warnings
warnings.filterwarnings('ignore')

In [5]:
media = 'digital'

SHAREPOINT_DATA_PATH = media_args[media]['sharepoint']['data']
SHAREPOINT_MAPPING_PATH = media_args[media]['sharepoint']['mapping']
SHAREPOINT_QA_PATH = media_args[media]['sharepoint']['qa']
SHAREPOINT_FLAT_PATH = media_args[media]['sharepoint']['flat']
REDSHIFT_METRICS_QUERY = media_args[media]['redshift']['all_metrics']
REDSHIFT_METRICS_FILENAME = media_args[media]['redshift']['metric_filename']
REDSHIFT_METRICS_SHEETNAME = media_args[media]['redshift']['metric_sheetname']
REDSHIFT_QA_QUERY = media_args[media]['redshift']['qa_qry']

**Connect to SharePoint**

In [6]:
app = SharePoint('AmericanExpressUSGABM')

app.list_contents(SHAREPOINT_MAPPING_PATH)
app.download_files(ASSETS_PATH)

connected to: https://interpublic.sharepoint.com/sites/AmericanExpressUSGABM

listing files from: Measurement%20%20Analytics%20Folder/GABM/Global%20Analytics/DigitalQA/Digital/02Mapping

downloading data to: C:\Users\carmelo.urena\Documents\Main\OneDrive_BAK\Amex\DigitalQA\SharePoint\sharepoint_api\assets



**Get All Metrics from DB**

In [7]:
# import allmetrics from redhshift
df_rs_metrics = run_qry(REDSHIFT_METRICS_QUERY)

**Compare Metrics**

In [10]:
# import current state
pth = glob.glob(os.path.join(ASSETS_PATH, f"*{REDSHIFT_METRICS_FILENAME}*"))[0]
df_cur_metrics = pd.read_excel(pth, sheet_name=REDSHIFT_METRICS_SHEETNAME)

In [12]:
# get list of metrics
metric_field = 'universal_metric' if media=='social' else 'metric'
rs_metrics = list(df_rs_metrics[metric_field])
cur_metrics = list(df_cur_metrics[metric_field])

new_metrics = [m for m in rs_metrics if not(m in cur_metrics)]

# if there is missing data, create a df
if len(new_metrics) > 0:
    print(f"new metrics identified: {len(new_metrics)}")
    
    # initiate openpyxl
    wb = openpyxl.load_workbook(pth)
    ws = wb[REDSHIFT_METRICS_SHEETNAME]
    
    starting_row = ws.max_row + 1
    
    if media in ['search', 'digital']:
        for i in range(len(new_metrics)):
            ws.cell(starting_row + i, 1).value = int(SharePoint.dt)
            ws.cell(starting_row + i, 2).value = new_metrics[i]
    elif media=='social':
        rs_metrics_full = df_rs_metrics[df_rs_metrics['universal_metric'].isin(new_metrics)]
        for i in range(len(new_metrics)):
            ws.cell(starting_row + i, 1).value = int(SharePoint.dt)
            ws.cell(starting_row + i, 2).value = rs_metrics_full.iloc[i, 0]
            ws.cell(starting_row + i, 3).value = rs_metrics_full.iloc[i, 1]
            ws.cell(starting_row + i, 4).value = rs_metrics_full.iloc[i, 2]
    
        
    wb.save(pth)
    wb.close()
    
    app.upload_files(SHAREPOINT_MAPPING_PATH, pth)
else:
    print('no metrics to update')

new metrics identified: 4656
uploading data
from: C:\Users\carmelo.urena\Documents\Main\OneDrive_BAK\Amex\DigitalQA\SharePoint\sharepoint_api\assets\Digital_mapping.xlsx
to: Measurement%20%20Analytics%20Folder/GABM/Global%20Analytics/DigitalQA/Digital/02Mapping



# Function

In [16]:
def metric_update():
    '''Download metric mapping from SharePoint and append new metrics from RedShift database.'''
    
    # connect to SharePoint
    app = SharePoint('AmericanExpressUSGABM')
    app.list_contents(SHAREPOINT_MAPPING_PATH)
    app.download_files(ASSETS_PATH)
    
    # get metrics from database
    df_rs_metrics = run_qry(REDSHIFT_METRICS_QUERY)
    
    # import the SharePoint metric file
    pth = glob.glob(os.path.join(ASSETS_PATH, f"*{REDSHIFT_METRICS_FILENAME}*"))[0]
    df_cur_metrics = pd.read_excel(pth, sheet_name=REDSHIFT_METRICS_SHEETNAME)
    
    # get list of metrics
    metric_field = 'universal_metric' if media=='social' else 'metric'
    rs_metrics = list(df_rs_metrics[metric_field])
    cur_metrics = list(df_cur_metrics[metric_field])

    new_metrics = [m for m in rs_metrics if not(m in cur_metrics)]

    # if there is missing data, create a df
    if len(new_metrics) > 0:
        print(f"new metrics identified: {len(new_metrics)}")

        # initiate openpyxl
        wb = openpyxl.load_workbook(pth)
        ws = wb[REDSHIFT_METRICS_SHEETNAME]

        starting_row = ws.max_row + 1

        if media=='search':
            for i in range(len(new_metrics)):
                ws.cell(starting_row + i, 1).value = int(SharePoint.dt)
                ws.cell(starting_row + i, 2).value = new_metrics[i]
        elif media=='social':
            rs_metrics_full = df_rs_metrics[df_rs_metrics['universal_metric'].isin(new_metrics)]
            for i in range(len(new_metrics)):
                ws.cell(starting_row + i, 1).value = int(SharePoint.dt)
                ws.cell(starting_row + i, 2).value = rs_metrics_full.iloc[i, 0]
                ws.cell(starting_row + i, 3).value = rs_metrics_full.iloc[i, 1]
                ws.cell(starting_row + i, 4).value = rs_metrics_full.iloc[i, 2]


        wb.save(pth)
        wb.close()

        app.upload_files(SHAREPOINT_MAPPING_PATH, pth)
    else:
        print('no metrics to update')